In [1]:
!pip install transformers
!pip install py_vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=2a89ff132673802dc947e539de626e461c9909e1192674297857bdba5592adaf
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [4]:
import re
import py_vncorenlp

py_vncorenlp.download_model(save_dir='./')
segment_model = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/DS105/model_save')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/DS105/model_save')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030" "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_special_characters(text):
    pattern = r"[^\w\s.,;:?]"
    return re.sub(pattern, "", text)

def from_logit_to_label(logit):
  return 'Positive' if logit == 0 else 'Negative'

In [35]:
def classification_review(text: list, tokenizer=tokenizer, model=model):
  input_ids, attention_masks = [], []
  sentences = text.copy()

  for i in range(len(text)):
    text[i] = text[i].replace('\n', '. ')
    text[i] = remove_emoji(text[i])
    text[i] = remove_special_characters(text[i])
    text[i] = segment_model.word_segment(text[i])[0]

    encoded_dict = tokenizer.encode_plus(
        text[i],
        max_length=100,
        padding='max_length',
        return_attention_mask=True,
        truncation=True
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)

  dataset = TensorDataset(input_ids, attention_masks)
  dataloader = DataLoader(dataset, shuffle=False, batch_size=32)

  model.eval()

  for batch in dataloader:
    b_input_ids, b_attention_mask = batch

    with torch.no_grad():
      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_attention_mask)

  logits =result.logits

  labels_out = torch.argmax(logits, dim=1)

  for i, sent in enumerate(sentences):
    print("{} : {}".format(sent, from_logit_to_label(labels_out[i])))

  return labels_out

In [40]:
res = classification_review(['Hàng hóa tệ'])

Hàng hóa tệ : Negative
